# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
# Monte the files from MyDrive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# install packages for hugging face
!pip install sentence-transformers faiss-cpu tqdm nltk datasets lemminflect spacy



^C


   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.8/15.0 MB 9.2 MB/s eta 0:00:02
   ------------- -------------------------- 5.0/15.0 MB 13.1 MB/s eta 0:00:01
   --------------------- ------------------ 8.1/15.0 MB 14.0 MB/s eta 0:00:01
   --------------------------- ------------ 10.5/15.0 MB 13.4 MB/s eta 0:00:01
   -------------------------------- ------- 12.3/15.0 MB 12.3 MB/s eta 0:00:01
   ------------------------------------- -- 14.2/15.0 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 15.0/15.0 MB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/769.7 kB ? eta -:--:--
   ---------------------------------------- 769.7/769.7 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
    --------------------------------------- 3.1/212.5 MB 16.8 MB/s eta 0:00:13
   - -------------------------------------- 6.6/212.5 MB 16.8 MB/s eta 0:00:13
 

In [1]:
# Import all the packages
import numpy as np
import json
from collections import Counter
from sentence_transformers import SentenceTransformer
import pickle
import faiss
from tqdm import tqdm
import nltk
import re
from datasets import Dataset
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
# Analyse train-claims and evidences

train_claims_path = './data/train-claims.json'
dev_claims_path = './data/dev-claims.json'
evidence_path = './data/evidence.json'

def analyze_data(train_claims_path, dev_claims_path, evidence_path):
  # Load the datasets
  with open(train_claims_path, 'r') as f:
    train_claims = json.load(f)
  with open(dev_claims_path, 'r') as f:
    dev_claims = json.load(f)
  with open(evidence_path, 'r') as f:
    evidences = json.load(f)

  #  Analyze train-claims.json
  print("=== Train Claims Analysis ===")
  print(f"Total train claims: {len(train_claims)}")
  # Claim length (in characters and tokens) & evidence count
  claim_lengths_char = []
  claim_lengths_token = []
  evidence_counts = []
  claim_labels = Counter()

  for claim in train_claims.values():
    claim_text = claim["claim_text"]
    claim_lengths_char.append(len(claim_text))
    claim_lengths_token.append(len(claim_text.split()))
    evidence_counts.append(len(claim.get("evidences", [])))
    claim_labels[claim["claim_label"]] += 1

  print(f"Max claim length (characters): {max(claim_lengths_char)}")
  print(f"Min claim length (characters): {min(claim_lengths_char)}")
  print(f"Mean claim length (characters): {np.mean(claim_lengths_char):.1f}")
  print(f"Max claim length (tokens): {max(claim_lengths_token)}")
  print(f"Min claim length (tokens): {min(claim_lengths_token)}")
  print(f"Mean claim length (tokens): {np.mean(claim_lengths_token):.1f}")

  print(f"Max evidence count: {max(evidence_counts)}")
  print(f"Min evidence count: {min(evidence_counts)}")
  print(f"Mean evidence count: {np.mean(evidence_counts):.2f}")

  print("Claim label distribution:")
  for label, count in claim_labels.items():
    print(f"  {label}: {count} ({count / len(train_claims):.2%})")

  # Analyze dev-claims.json
  print("=== Dev Claims Analysis ===")
  print(f"Total dev claims: {len(dev_claims)}")
  # Claim length (in characters and tokens) & evidence count
  claim_lengths_char = []
  claim_lengths_token = []
  evidence_counts = []
  claim_labels = Counter()

  for claim in dev_claims.values():
    claim_text = claim["claim_text"]
    claim_lengths_char.append(len(claim_text))
    claim_lengths_token.append(len(claim_text.split()))
    evidence_counts.append(len(claim.get("evidences", [])))
    claim_labels[claim["claim_label"]] += 1

  print(f"Max claim length (characters): {max(claim_lengths_char)}")
  print(f"Min claim length (characters): {min(claim_lengths_char)}")
  print(f"Mean claim length (characters): {np.mean(claim_lengths_char):.1f}")
  print(f"Max claim length (tokens): {max(claim_lengths_token)}")
  print(f"Min claim length (tokens): {min(claim_lengths_token)}")
  print(f"Mean claim length (tokens): {np.mean(claim_lengths_token):.1f}")

  print(f"Max evidence count: {max(evidence_counts)}")
  print(f"Min evidence count: {min(evidence_counts)}")
  print(f"Mean evidence count: {np.mean(evidence_counts):.2f}")

  print("Claim label distribution:")
  for label, count in claim_labels.items():
    print(f"  {label}: {count} ({count / len(dev_claims):.2%})")
  # Analyze evidences.json
  print("\n=== Evidence Analysis ===")
  print(f"Total evidences: {len(evidences)}")
  evidence_lengths_char = [len(text) for text in evidences.values()]
  evidence_lengths_token = [len(text.split()) for text in evidences.values()]

  print(f"Max evidence length (characters): {max(evidence_lengths_char)}")
  print(f"Min evidence length (characters): {min(evidence_lengths_char)}")
  print(f"Mean evidence length (characters): {np.mean(evidence_lengths_char):.1f}")
  print(f"Max evidence length (tokens): {max(evidence_lengths_token)}")
  print(f"Min evidence length (tokens): {min(evidence_lengths_token)}")
  print(f"Mean evidence length (tokens): {np.mean(evidence_lengths_token):.1f}")

# Example usage:
analyze_data(train_claims_path, dev_claims_path, evidence_path)


=== Train Claims Analysis ===
Total train claims: 1228
Max claim length (characters): 332
Min claim length (characters): 26
Mean claim length (characters): 123.0
Max claim length (tokens): 67
Min claim length (tokens): 4
Mean claim length (tokens): 20.1
Max evidence count: 5
Min evidence count: 1
Mean evidence count: 3.36
Claim label distribution:
  DISPUTED: 124 (10.10%)
  REFUTES: 199 (16.21%)
  SUPPORTS: 519 (42.26%)
  NOT_ENOUGH_INFO: 386 (31.43%)
=== Dev Claims Analysis ===
Total dev claims: 154
Max claim length (characters): 406
Min claim length (characters): 31
Mean claim length (characters): 128.0
Max claim length (tokens): 65
Min claim length (tokens): 4
Mean claim length (tokens): 21.1
Max evidence count: 5
Min evidence count: 1
Mean evidence count: 3.19
Claim label distribution:
  SUPPORTS: 68 (44.16%)
  NOT_ENOUGH_INFO: 41 (26.62%)
  REFUTES: 27 (17.53%)
  DISPUTED: 18 (11.69%)

=== Evidence Analysis ===
Total evidences: 1208827
Max evidence length (characters): 3148
Min ev

In [2]:
train_claims_path = './data/train-claims.json'
dev_claims_path = './data/dev-claims.json'
evidence_path = './data/evidence.json'

In [3]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import json
import random

# 路径设定
train_claims_path = './data/train-claims.json'
evidence_path = './data/evidence.json'

# 加载数据
with open(train_claims_path, 'r') as f:
    train_claims = json.load(f)
with open(evidence_path, 'r') as f:
    evidence_dict = json.load(f)

# 构建训练样本列表（claim, evidence_text） -> label默认为1.0
train_samples = []
missed = 0

for claim in train_claims.values():
    claim_text = claim['claim_text']
    evidence_ids = claim.get('evidences', [])
    for eid in evidence_ids:
        if eid in evidence_dict:
            ev_text = evidence_dict[eid]
            train_samples.append(InputExample(texts=[claim_text, ev_text], label=1.0))
        else:
            missed += 1

print(f"Total training pairs: {len(train_samples)}")
print(f"Missing evidence ids: {missed}")

# 加载预训练模型
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 构建 Dataloader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=32)

# 定义损失函数
train_loss = losses.MultipleNegativesRankingLoss(model)

# 开始训练
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,  # 可设为 2-3，看你资源
    warmup_steps=100,
    show_progress_bar=True
)

# 保存模型
model.save('./model/my_finetuned_minilm_retriever')
print("✅ Finetuned model saved.")


Total training pairs: 4122
Missing evidence ids: 0


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


✅ Finetuned model saved.


In [4]:
model = SentenceTransformer('./model/my_finetuned_minilm_retriever')

In [5]:
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from lemminflect import getAllInflections
nlp = spacy.load("en_core_web_sm")
with open(train_claims_path, 'r') as f:
    train_claims = json.load(f)

with open(evidence_path, 'r') as f:
    evidence_dict = json.load(f)

# === 从 claim_text 中提取所有名词并统计词频 ===
all_nouns = []
for claim_obj in train_claims.values():
    doc = nlp(claim_obj["claim_text"])
    nouns = [token.lemma_.lower() for token in doc if token.pos_ == "NOUN"]
    all_nouns.extend(nouns)

# === 选出 top 100 高频名词作为关键词 ===
top_keywords = set(word for word, _ in Counter(all_nouns).most_common(100))

all_forms = set()
for lemma in top_keywords:
  all_forms.add(lemma)
  # 拿到所有可能的名词形式
  infl_map = getAllInflections(lemma, upos="NOUN")
  # infl_map 是个 dict：{ 'NNS': ['cats'], 'NNPS': ['children'], ... }
  for forms in infl_map.values():
      all_forms.update(forms)


In [6]:
def contains_climate_keywords(text: str, all_forms: set) -> bool:
    # 直接把文本小写化、拆成“词”后查集合
    words = re.findall(r"\b[a-z']+\b", text.lower())
    return any(word in all_forms for word in words)


def is_english(text: str, threshold: float = 0.5) -> bool:
  # 清理文本，仅保留字母和空格
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  if len(text) == 0:  # 如果文本清理后为空，返回False
    return False
  # 计算英文字符占比
  english_char_count = sum(1 for char in text if char.isalpha())
  return (english_char_count / len(text)) >= threshold

def clean_and_split(eid, text):
  result_ids = []
  result_texts = []
  sentences = sent_tokenize(text)
  for i, sent in enumerate(sentences):
    sent = sent.lower()
    sent = re.sub(r'[^a-z0-9\s]', '', sent)  # 去除标点
    sent = re.sub(r'\s+', ' ', sent).strip()
    if len(sent.split()) >= 5:  # 可选：过滤太短的文本
      result_ids.append(f"{eid}_s{i}")  # 用原始的 eID 加上句子索引
      result_texts.append(sent)
  return result_ids, result_texts



In [16]:
with open(evidence_path, 'r') as f:
  evidence_dict = json.load(f)
# ——— 1. 批量剔除非英文 —— #
eids  = list(evidence_dict.keys())
texts = list(evidence_dict.values())
english_pairs = [
  (eid, txt)
  for eid, txt in zip(eids, texts)
  if is_english(txt)
]
print(f"Step1: English keep {len(english_pairs)}/{len(texts)}")

# ——— 2. 批量剔除非 climate-related —— #
climate_pairs = [
    (eid, txt)
    for eid, txt in english_pairs
    if contains_climate_keywords(txt, all_forms)
]
print(f"Step2: Climate-related keep {len(climate_pairs)}/{len(english_pairs)}")


Step1: English keep 1207838/1208827
Step2: Climate-related keep 385471/1207838


In [17]:
# 构建筛选后的 evidence ID 集合
filtered_evidence_ids = set(eid for eid, _ in climate_pairs)

# 初始化计数器
total_ground_truth = 0
total_retrieved = 0
with open(dev_claims_path, 'r') as f:
  dev_claims = json.load(f)
for claim_data in dev_claims.values():
    true_evidences = claim_data.get("evidences", [])
    total_ground_truth += len(true_evidences)
    total_retrieved += sum(1 for eid in true_evidences if eid in filtered_evidence_ids)

# 避免除以 0 的错误
if total_ground_truth == 0:
    overall_recall = 0.0
else:
    overall_recall = total_retrieved / total_ground_truth

print(f"Overall Recall: {overall_recall:.2%}")


Overall Recall: 93.08%


In [7]:
encode = True # Variable to control whether encode the evidence
# Load the sentence-BERT (all-MiniLM-L6-v2)

if (encode):
  with open(evidence_path, 'r') as f:
    evidence_dict = json.load(f)
  # ——— 1. 批量剔除非英文 —— #
  eids  = list(evidence_dict.keys())
  texts = list(evidence_dict.values())
  english_pairs = [
    (eid, txt)
    for eid, txt in zip(eids, texts)
    if is_english(txt)
  ]
  print(f"Step1: English keep {len(english_pairs)}/{len(texts)}")

  # ——— 2. 批量剔除非 climate-related —— #
  climate_pairs = [
    (eid, txt)
    for eid, txt in english_pairs
    if contains_climate_keywords(txt, all_forms)
  ]
  print(f"Step2: Climate-related keep {len(climate_pairs)}/{len(english_pairs)}")


  # 清洗和分句后的结果
  cleaned_evidence_ids = []
  cleaned_evidence_texts = []
  original_evidence_ids = []  # 记录每个分句所属的原始 evidence_id

  # 遍历 evidence 数据并进行清洗和分句
  for eid, text in climate_pairs:
    cleaned_ids, cleaned_texts = clean_and_split(eid, text)
    cleaned_evidence_ids.extend(cleaned_ids)
    cleaned_evidence_texts.extend(cleaned_texts)
    original_evidence_ids.extend([eid] * len(cleaned_ids))  # 每个分句都记录原始的 eID

  # 使用 Sentence-BERT 对清洗后的分句进行编码
  evidence_embeddings = model.encode(
    cleaned_evidence_texts,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
  )

  # 保存编码后的嵌入（embeddings）
  np.save("./word_embedding/evidence_embeddings.npy", evidence_embeddings)

  # 保存分句后的 evidence_ids 和 texts 以及对应的原始 evidence_id 映射
  with open("./word_embedding/evidence_meta.pkl", "wb") as f:
      pickle.dump((cleaned_evidence_ids, cleaned_evidence_texts, original_evidence_ids), f)


Step1: English keep 1207838/1208827
Step2: Climate-related keep 385471/1207838


Batches:   0%|          | 0/12133 [00:00<?, ?it/s]

In [10]:
# Load numpy embeddings
evidence_embeddings = np.load("./word_embedding/evidence_embeddings.npy")

# Load evidence_ids, evidence_texts, and original_evidence_ids
with open("./word_embedding/evidence_meta.pkl", "rb") as f:
  evidence_ids, evidence_texts, original_evidence_ids = pickle.load(f)


dimension = evidence_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product = Cosine similarity if normalized
index.add(evidence_embeddings)


def clean_claim(claim: str) -> str:
  # 小写化
  claim = claim.lower()
  # 去除标点符号
  claim = re.sub(r'[^a-z0-9\s]', '', claim)
  # 去除多余空格
  claim = re.sub(r'\s+', ' ', claim).strip()
  # 如果需要，你也可以添加去除停用词的步骤
  # claim = " ".join([word for word in claim.split() if word not in stopwords])
  return claim

# Create mapping
evidence_dict = dict(zip(evidence_ids, evidence_texts))

def retrieve_evidence(claim_text, top_k=5):
  # 清洗 claim
  cleaned_claim = clean_claim(claim_text)
  # 编码 claim
  claim_embedding = model.encode([cleaned_claim], convert_to_numpy=True, normalize_embeddings=True)
  # 执行检索
  scores, indices = index.search(claim_embedding, top_k*10)

  # 准备去重筛选
  seen_original_ids = set()
  results = []

  for i in indices[0]:
    eid = evidence_ids[i]
    text = evidence_dict[eid]
    original_id = original_evidence_ids[i]

    if original_id not in seen_original_ids:
      results.append((original_id, eid, text))
      seen_original_ids.add(original_id)

    if len(results) >= top_k:
      break

  return results

In [13]:
from tqdm import tqdm
# Evaluate the evidence retrieval method
with open(dev_claims_path, 'r') as f:
  dev_claims = json.load(f)

claim_ids = list(dev_claims.keys())
top_k = 3

recalls = []
precisions = []
f1s = []

for cid in tqdm(claim_ids, desc="Evaluating"):
  truth = set(dev_claims[cid]["evidences"])
  retrieved = [original_id for original_id, _, _ in retrieve_evidence(dev_claims[cid]["claim_text"], top_k)]

  hit = len(truth & set(retrieved))

  # Recall
  recall = hit / len(truth) if len(truth) > 0 else 0
  recalls.append(recall)

  # Precision
  precision = hit / top_k if top_k > 0 else 0
  precisions.append(precision)

  # F1
  if precision + recall > 0:
    f1 = 2 * precision * recall / (precision + recall)
  else:
    f1 = 0
  f1s.append(f1)

# Output evaluation metrics
print(f"\nAverage Recall@{top_k}:   {np.mean(recalls):.2%}")
print(f"Average Precision@{top_k}:{np.mean(precisions):.2%}")
print(f"Average F1@{top_k}:       {np.mean(f1s):.2%}")

Evaluating: 100%|██████████| 154/154 [00:05<00:00, 26.04it/s]


Average Recall@3:   20.08%
Average Precision@3:18.40%
Average F1@3:       17.77%


In [23]:
claim = "[South Australia] has the most expensive electricity in the world."
with open(evidence_path, 'r') as f:
  evidences = json.load(f)
retrieved = retrieve_evidence(claim, top_k=7)

for org_eid,_,_ in retrieved:
  print(f"{org_eid}: {evidences[org_eid]}\n")

evidence-67732: [citation needed] South Australia has the highest retail price for electricity in the country.

evidence-572512: "South Australia has the highest power prices in the world".

evidence-1172687: Germans in 2013 had some of the highest electricity costs in Europe.

evidence-169170: Germans in 2013 had some of the highest electricity costs in Europe.

evidence-699739: This year New Zealand produced 8,000 million kWh of electricity.

evidence-63059: As of November 2017, prices in the MENA Region (Middle East and North Africa) are at 7 cents per kWh or lower according to ACWA Power.

evidence-1170477: The USA consumes about 14% of the world total, using 90% of it for generation of electricity.



# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*

In [12]:
# 打开并加载训练数据和证据数据
with open(train_claims_path, 'r') as f:
    train_claims = json.load(f)

with open(evidence_path, 'r') as f:
    evidences = json.load(f)

# 打印前五个 claim 和对应的 evidence
for i, (claim_id, claim_data) in enumerate(list(train_claims.items())[:5]):  # 遍历前五个 claim
    claim_text = claim_data['claim_text']  # 获取 claim_text
    claim_label = claim_data['claim_label']  # 获取 claim_label
    # 获取该 claim 对应的 evidence
    corresponding_evidence = []
    for evidence_id in claim_data.get('evidences', []):  # 获取与该 claim 相关的 evidence_id 列表
        evidence = evidences.get(evidence_id, 'Evidence not found')
        corresponding_evidence.append(evidence)

    # 打印 claim 和其对应的 evidence
    print(f"Claim {i+1}:")
    print(f"Claim ID: {claim_id}")
    print(f"Claim Text: {claim_text}")
    print(f"Claim Label: {claim_label}")
    print("Corresponding Evidence:")
    for evidence in corresponding_evidence:
        print(f"  - {evidence}")
    print("\n" + "-"*50 + "\n")


Claim 1:
Claim ID: claim-1937
Claim Text: Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.
Claim Label: DISPUTED
Corresponding Evidence:
  - At very high concentrations (100 times atmospheric concentration, or greater), carbon dioxide can be toxic to animal life, so raising the concentration to 10,000 ppm (1%) or higher for several hours will eliminate pests such as whiteflies and spider mites in a greenhouse.
  - Plants can grow as much as 50 percent faster in concentrations of 1,000 ppm CO 2 when compared with ambient conditions, though this assumes no change in climate and no limitation on other nutrients.
  - Higher carbon dioxide concentrations will favourably affect plant growth and demand for water.

--------------------------------------------------

Claim 2:
Claim ID: claim-126
Claim Text: El Niño drove record highs in global temperatures suggesting rise may not be down to m